Environment: napari-sparrow

In [ ]:
%load_ext autoreload
%autoreload 2

### Import packages

In [ ]:
import napari_sparrow as nas
import sparrow as sp
import os
import spatialdata as sd
import numpy as np
import cv2 as cv2
from skimage import io

### Set paths and create folders

In [ ]:
# Specify path to root folder
root_folder = 'D:/Data/2023-07-CarolineAsselman-FIm/Analysis_v2'

# Specify ROI name

ROI = 'A1_ROI1'

# Path to input folder
images_path = os.path.join(root_folder, 'cropped_images', ROI)

# Path to output folders
output_path = os.path.join(root_folder, 'output', ROI, 'segmentation')
plots_path = os.path.join(output_path, 'plots')
masks_path = os.path.join(output_path, 'masks')
os.makedirs(output_path, exist_ok = True)
os.makedirs(plots_path, exist_ok = True)
os.makedirs(masks_path, exist_ok = True)

### Specify channels for segmentation

In [ ]:
from gene_lists import nucleus, RBC, immune
segmentation_channels = nucleus + RBC + immune

In [ ]:
input_list = []
c_coords_list = []

def create_input_lists(channel_list):
    for file in os.listdir(images_path):
        if (file.endswith('.tif') or file.endswith('tiff')) and any(channel in file for channel in channel_list):
            input_list.append(os.path.join(images_path, file))
            filename = os.path.splitext(file)[0]
            channel_name = [channel for channel in channel_list if channel in filename]
            c_coords_list.append(' '.join(channel_name))
        
create_input_lists(segmentation_channels)


### Create or read sdata

In [ ]:
sdata = sp.io.create_sdata(
    input= input_list,
    c_coords= c_coords_list,
    output_path=os.path.join(output_path, "sdata.zarr"),
    img_layer="raw_image",
    chunks=1024)
sdata

### Clean images

In [ ]:
for channel in segmentation_channels:
    sp.pl.plot_image(
        sdata, 
        img_layer='raw_image',
        output=os.path.join(plots_path , f'{channel}_0_raw.png'),
        channel=channel,
        vmin_img=0,
        vmax_img=65535, 
        figsize=(35,22.5)
    )

##### Histogram adjustment

In [ ]:
def histogram_adjustment(image, lower_channel_value: int = 100, higher_channel_value: int = 60000):
    image = image.compute()
    clipped_image = np.clip(image, lower_channel_value, higher_channel_value).astype(np.float64)
    new_image = np.clip(((clipped_image - lower_channel_value) / (higher_channel_value - lower_channel_value) * 65535), 0, 65535).astype(np.uint16)
    return new_image

In [ ]:
fn_kwargs={
    'DAPI': {"lower_channel_value": 6000, "higher_channel_value": 65535},
    'CD233_C-REA368': {"lower_channel_value": 600, "higher_channel_value": 25000},
    'CD15_C-VIMC6': {"lower_channel_value": 100, "higher_channel_value": 10000},
    'CD43_C-REA833': {"lower_channel_value": 1200, "higher_channel_value": 9000},
    'CD45_C-5B1': {"lower_channel_value": 200, "higher_channel_value": 15000},
    'CD45RA_C-REAL164': {"lower_channel_value": 300, "higher_channel_value": 12000},
    'CD45RB_C-REA119': {"lower_channel_value": 200, "higher_channel_value": 12000},
    'CD66b_C-REA306': {"lower_channel_value": 200, "higher_channel_value": 14000},
    'CD162_C-REA319': {"lower_channel_value": 800, "higher_channel_value": 4200},
    'HLADR_C-REAL550': {"lower_channel_value": 200, "higher_channel_value": 10000},
    'Syk_C-REA111': {"lower_channel_value": 200, "higher_channel_value": 3200},
    'CD57_C-REA769': {"lower_channel_value": 300, "higher_channel_value": 40000}
}

In [ ]:
sdata = sp.im.apply(
    sdata,
    histogram_adjustment,
    img_layer='raw_image',
    output_layer='hist',
    crd=None,
    chunks=None,
    overwrite=True,
    channel=segmentation_channels,
    fn_kwargs=fn_kwargs,
    combine_c=False
)

In [ ]:
for channel in segmentation_channels:
    sp.pl.plot_image(
        sdata, 
        img_layer='hist',
        output=os.path.join(plots_path , f'{channel}_1_hist.png'),
        channel=channel,
        vmin_img=0,
        vmax_img=65535, 
        figsize=(35,22.5)
    )

##### min/max filtering

In [ ]:
sdata = sp.im.min_max_filtering(
    sdata,
    img_layer="hist", 
    size_min_max_filter=85,
    output_layer="min_max_filtered", 
    overwrite=True)

In [ ]:
for channel in segmentation_channels:
    sp.pl.plot_image(
        sdata, 
        img_layer='min_max_filtered',
        output=os.path.join(plots_path , f'{channel}_2_min_max.png'),
        channel=channel,
        vmin_img=0,
        vmax_img=65535, 
        figsize=(35,22.5)
    )

##### contrast enhancement

In [ ]:
sdata = sp.im.enhance_contrast(
    sdata,
    img_layer="min_max_filtered", 
    contrast_clip=0.5,
    output_layer="clahe", 
    overwrite=True)

In [ ]:
for channel in segmentation_channels:
    sp.pl.plot_image(
        sdata, 
        img_layer='clahe',
        output=os.path.join(plots_path , f'{channel}_3_clahe.png'),
        channel=channel,
        vmin_img=0,
        vmax_img=65535, 
        figsize=(35,22.5)
    )

### Combine channels

##### Immune cells

In [ ]:
sdata = sp.im.combine(
    sdata, 
    img_layer="clahe", 
    output_layer="combined_immune", 
    nuc_channels=nucleus, 
    mem_channels=immune, 
    overwrite=True)

In [ ]:
sp.pl.plot_image(
    sdata, 
    img_layer='combined_immune',
    output=os.path.join(plots_path , f'combined_immune.png'),
    channel=1,
    vmin_img=0,
    vmax_img=180000, 
    figsize=(35,22.5)
)

In [ ]:
sdata = sp.im.min_max_filtering(
    sdata,
    img_layer="combined_immune", 
    size_min_max_filter=85,
    output_layer="combined_immune_min_max_filtered", 
    overwrite=True)

In [ ]:
sp.pl.plot_image(
    sdata, 
    img_layer='combined_immune_min_max_filtered',
    output=os.path.join(plots_path , f'combined_immune_min_max.png'),
    channel=1,
    vmin_img=0,
    vmax_img=180000, 
    figsize=(35,22.5)
)

##### RBC

In [ ]:
sdata = sp.im.combine(
    sdata, 
    img_layer="clahe", 
    output_layer="combined_RBC", 
    nuc_channels=nucleus, 
    mem_channels=RBC, 
    overwrite=True)

### Segmentation

#### Import segmentation model

In [ ]:
from sparrow.image.segmentation.segmentation_models._cellpose import _cellpose

#### Nucleus

##### Nucleus segmentation

In [ ]:
sdata = sp.im.segment( 
    sdata=sdata, 
    img_layer='combined_immune',
    output_labels_layer='segmentation_nucleus',
    output_shapes_layer='segmentation_nucleus_boundaries',
    device='cpu',
    model=_cellpose,
    min_size=80,
    flow_threshold=0.9,
    diameter=50,
    cellprob_threshold=-4,
    model_type='nuclei',
    chunks=2048,
    depth=(2*50,2*50),
    overwrite=False, #Should be set to False for the first time this cell is run. From then on it should be set to True.
    channels = [1, 0]
)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune', 
    channel = 0, 
    output=os.path.join(plots_path , 'segmentation_nucleus_0_DAPI.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune', 
    channel = 0, 
    shapes_layer="segmentation_nucleus_boundaries", 
    output=os.path.join(plots_path , 'segmentation_nucleus_1_unfiltered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

##### Filter nuclei based on size

In [ ]:
# Filter based on size
min_area = 250 # in pixels
max_area = 4000 # in pixels
sdata.shapes['segmentation_nucleus_filtered_boundaries'] = sdata.shapes['segmentation_nucleus_boundaries'][
    (sdata.shapes['segmentation_nucleus_boundaries'].area >= min_area) &
    (sdata.shapes['segmentation_nucleus_boundaries'].area <= max_area)
]
sd.save_transformations(sdata)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune', 
    channel = 0, 
    shapes_layer="segmentation_nucleus_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_nucleus_2_filtered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

##### Convert filtered shapes layer into labels layer

In [ ]:
import rasterio.features
from affine import Affine

def create_labels_from_shapes(sdata, shapes_layer: str, out_shape: list, output_layer: str):
    
    transform = Affine.translation(0, 0)
    
    masks = rasterio.features.rasterize(
        zip(
            sdata[shapes_layer].geometry,
            sdata[shapes_layer].index.values.astype(float),
        ),
        out_shape=out_shape,
        dtype="uint32",
        fill=0,
        transform=transform
    )

    sdata = sp.im._add_label_layer(sdata, arr=masks, chunks=1024, output_layer=output_layer)

    return sdata

In [ ]:
sdata = create_labels_from_shapes(
    sdata,
    shapes_layer = 'segmentation_nucleus_filtered_boundaries',
    out_shape = [4500, 7000],
    output_layer = 'segmentation_nucleus_filtered'
)

##### Expand nuclei

In [ ]:
sdata=sp.im.expand_labels_layer(
        sdata,
        labels_layer="segmentation_nucleus_filtered",
        distance=10,
        depth=200,
        chunks=5000,
        output_labels_layer="segmentation_nucleus_expanded",
        output_shapes_layer="segmentation_nucleus_expanded_boundaries",
        overwrite=True,
    )

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune', 
    channel = 0, 
    shapes_layer="segmentation_nucleus_expanded_boundaries", 
    output=os.path.join(plots_path , 'segmentation_nucleus_3_expanded.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

#### Immune cells

#### Cell segmentation

In [ ]:
sdata = sp.im.segment( 
    sdata=sdata, 
    img_layer='combined_immune_min_max_filtered',
    output_labels_layer='segmentation_immune_whole_cell',
    output_shapes_layer='segmentation_immune_whole_cell_boundaries',
    device='cpu',
    model=_cellpose,
    min_size=80,
    flow_threshold=0.9,
    diameter=50,
    cellprob_threshold=-3,
    model_type='cyto',
    chunks=2048,
    depth=(2*50,2*50),
    overwrite=True, #Should be set to False for the first time this cell is run. From then on it should be set to True.
    channels = [2, 1]
)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 0, 
    output=os.path.join(plots_path , 'segmentation_immune_0_DAPI.png'),
    vmin_img=0,
    vmax_img=65535,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 0, 
    shapes_layer="segmentation_immune_whole_cell_boundaries", 
    output=os.path.join(plots_path , 'segmentation_immune_1_DAPI_unfiltered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 1, 
    output=os.path.join(plots_path , 'segmentation_immune_2_protein.png'),
    vmin_img=0,
    vmax_img=170000,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 1, 
    shapes_layer="segmentation_immune_whole_cell_boundaries", 
    output=os.path.join(plots_path , 'segmentation_immune_3_protein_unfiltered.png'),
    vmin_img=0,
    vmax_img=170000,
    alpha=0.2,
    figsize=(35,22.5))

##### Filter immune cells based on size

In [ ]:
# Filter based on size
min_area = 250 # in pixels
max_area = 5500 # in pixels
sdata.shapes['segmentation_immune_cell_size_filtered_boundaries'] = sdata.shapes['segmentation_immune_whole_cell_boundaries'][
    (sdata.shapes['segmentation_immune_whole_cell_boundaries'].area >= min_area) &
    (sdata.shapes['segmentation_immune_whole_cell_boundaries'].area <= max_area)
]
sd.save_transformations(sdata)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 1, 
    shapes_layer="segmentation_immune_cell_size_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_immune_4_size_filtered.png'),
    vmin_img=0,
    vmax_img=170000,
    alpha=0.2,
    figsize=(35,22.5))

##### Filter immune cells based on intensity

In [ ]:
sdata = sp.tb.allocate_intensity(
        sdata, img_layer="combined_immune_min_max_filtered", labels_layer="segmentation_immune_whole_cell", channels=1, chunks=4000)

sdata = sp.tb.add_regionprop_features(
        sdata, labels_layer="segmentation_immune_whole_cell", append_labels_layer_name=False)

In [ ]:
# Filter based on intensity
min_intensity = 13000
df_immune_filtered = (sdata.table.to_df()['1_segmentation_immune_whole_cell'] / sdata.table.obs['area'])> min_intensity
index_list = df_immune_filtered.index[df_immune_filtered].tolist()

sdata.shapes['segmentation_immune_cell_intensity_filtered_boundaries'] = sdata.shapes['segmentation_immune_cell_size_filtered_boundaries'][
    sdata.shapes['segmentation_immune_cell_size_filtered_boundaries'].index.isin(index_list)
]
sd.save_transformations(sdata)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune', 
    channel = 1, 
    shapes_layer="segmentation_immune_cell_intensity_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_immune_5_intensity_filtered.png'),
    vmin_img=0,
    vmax_img=170000,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
for channel in immune:
    sp.pl.plot_shapes(
        sdata, 
        img_layer='hist', 
        channel = channel, 
        shapes_layer="segmentation_immune_cell_intensity_filtered_boundaries", 
        output=os.path.join(plots_path , f'segmentation_immune_6_intensity_filtered_{channel}.png'),
        vmin_img=0,
        vmax_img=65535,
        alpha=0.2,
        figsize=(35,22.5))

##### Convert filtered shapes layer into labels layer

In [ ]:
sdata = create_labels_from_shapes(
    sdata,
    shapes_layer = 'segmentation_immune_cell_intensity_filtered_boundaries',
    out_shape = [4500, 7000],
    output_layer = 'segmentation_immune_cell_intensity_filtered'
)

#### RBC

##### Cell segmentation

In [ ]:
sdata = sp.im.segment( 
    sdata=sdata, 
    img_layer='combined_RBC',
    output_labels_layer='segmentation_RBC_cell',
    output_shapes_layer='segmentation_RBC_cell_boundaries',
    device='cpu',
    model=_cellpose,
    min_size=80,
    flow_threshold=0.9,
    diameter=50,
    cellprob_threshold=-3,
    model_type='cyto',
    chunks=2048,
    depth=(2*50,2*50),
    overwrite=False, #Should be set to False for the first time this cell is run. From then on it should be set to True.
    channels = [2, 0]
)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_RBC', 
    channel = 1, 
    output=os.path.join(plots_path , 'segmentation_RBC_0_protein.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_RBC', 
    channel = 1, 
    shapes_layer="segmentation_RBC_cell_boundaries", 
    output=os.path.join(plots_path , 'segmentation_RBC_1_unfiltered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

##### Filter RBC based on size

In [ ]:
# Filter based on size
min_area = 250 # in pixels
max_area = 2000 # in pixels
sdata.shapes['segmentation_RBC_cell_size_filtered_boundaries'] = sdata.shapes['segmentation_RBC_cell_boundaries'][
    (sdata.shapes['segmentation_RBC_cell_boundaries'].area >= min_area) &
    (sdata.shapes['segmentation_RBC_cell_boundaries'].area <= max_area)
]
sd.save_transformations(sdata)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_RBC', 
    channel = 1, 
    shapes_layer="segmentation_RBC_cell_size_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_RBC_2_size_filtered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

##### Filter RBC based on intensity

In [ ]:
sdata = sp.tb.allocate_intensity(
        sdata, img_layer="combined_RBC", labels_layer="segmentation_RBC_cell", channels=1, chunks=4000)

sdata = sp.tb.add_regionprop_features(
        sdata, labels_layer="segmentation_RBC_cell", append_labels_layer_name=False)

In [ ]:
# Filter based on intensity
min_intensity = 3000
df_RBC_filtered = (sdata.table.to_df()['1_segmentation_RBC_cell'] / sdata.table.obs['area'])> min_intensity
index_list = df_RBC_filtered.index[df_RBC_filtered].tolist()

sdata.shapes['segmentation_RBC_cell_intensity_filtered_boundaries'] = sdata.shapes['segmentation_RBC_cell_size_filtered_boundaries'][
    sdata.shapes['segmentation_RBC_cell_size_filtered_boundaries'].index.isin(index_list)
]
sd.save_transformations(sdata)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_RBC', 
    channel = 1, 
    shapes_layer="segmentation_RBC_cell_intensity_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_RBC_3_intensity_filtered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

##### Convert filtered shapes layer into labels layer

In [ ]:
sdata = create_labels_from_shapes(
    sdata,
    shapes_layer = 'segmentation_RBC_cell_intensity_filtered_boundaries',
    out_shape = [4500, 7000],
    output_layer = 'segmentation_RBC_cell_intensity_filtered'
)

### Merge segmentation masks

In [ ]:
sdata=sp.im.merge_labels_layers(
    sdata,
    labels_layer_1="segmentation_immune_cell_intensity_filtered",
    labels_layer_2="segmentation_RBC_cell_intensity_filtered",
    depth=200,
    chunks=2048,
    output_labels_layer="merged_immune_RBC",
    output_shapes_layer="merged_immune_RBC_boundaries",
    overwrite=True,
)

# NOTE: This code will give priority to the immune cell masks over the RBC masks (i.e.: if there is a collision, the immune cell will preserve its masks, while the RBC masks will be adapted)

In [ ]:
sdata = sp.im.merge_labels_layers_nuclei(
    sdata,
    labels_layer="merged_immune_RBC",
    labels_layer_nuclei_expanded="segmentation_nucleus_expanded",
    labels_layer_nuclei="segmentation_nucleus_filtered",
    output_labels_layer="merged_all",
    output_shapes_layer="merged_all_boundaries",
    overwrite=True,
    chunks=2048,
    depth=200,
)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 0, 
    shapes_layer="merged_all_boundaries", 
    output=os.path.join(plots_path , 'segmentation_merged_DAPI.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 1, 
    shapes_layer="merged_all_boundaries", 
    output=os.path.join(plots_path , 'segmentation_merged_immune.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_RBC', 
    channel = 1, 
    shapes_layer="merged_all_boundaries", 
    output=os.path.join(plots_path , 'segmentation_merged_RBC.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

##### Filter based on size

In [ ]:
# Filter based on size
min_area = 250 # in pixels
max_area = 8500 # in pixels
sdata.shapes['merged_all_filtered_boundaries'] = sdata.shapes['merged_all_boundaries'][
    (sdata.shapes['merged_all_boundaries'].area >= min_area) &
    (sdata.shapes['merged_all_boundaries'].area <= max_area)
]
sd.save_transformations(sdata)

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 0, 
    shapes_layer="merged_all_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_merged_DAPI_filtered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune_min_max_filtered', 
    channel = 1, 
    shapes_layer="merged_all_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_merged_immune_filtered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_RBC', 
    channel = 1, 
    shapes_layer="merged_all_filtered_boundaries", 
    output=os.path.join(plots_path , 'segmentation_merged_RBC_filtered.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

##### Convert filtered shapes layer into labels layer

In [ ]:
sdata = create_labels_from_shapes(
    sdata,
    shapes_layer = 'merged_all_filtered_boundaries',
    out_shape = [4500, 7000],
    output_layer = 'merged_all_filtered'
)

### Align labels layers

In [ ]:
sdata = sp.im.align_labels_layers(
        sdata,
        labels_layer_1="segmentation_nucleus_filtered",
        labels_layer_2="merged_all_filtered",
        output_labels_layer="segmentation_nucleus_aligned",
        output_shapes_layer="segmentation_nucleus_aligned_boundaries",
        overwrite=True,
        chunks=256,
        depth=100,
    )

In [ ]:
sp.pl.plot_shapes(
    sdata, 
    img_layer='combined_immune', 
    channel = 'DAPI', 
    shapes_layer="segmentation_nucleus_aligned_boundaries", 
    output=os.path.join(plots_path , 'segmentation_nucleus_4_aligned.png'),
    vmin_img=0,
    vmax_img=65535,
    alpha=0.2,
    figsize=(35,22.5))

### Save segmentation masks

In [ ]:
io.imsave(os.path.join(masks_path, 'segmentation_mask_merged.tiff'), sdata.labels['merged_all_filtered'].compute())
io.imsave(os.path.join(masks_path, 'segmentation_mask_nucleus.tiff'), sdata.labels['segmentation_nucleus_aligned'].compute())
io.imsave(os.path.join(masks_path, 'segmentation_mask_expanded_nucleus.tiff'), sdata.labels['segmentation_nucleus_expanded'].compute())
io.imsave(os.path.join(masks_path, 'segmentation_mask_immune.tiff'), sdata.labels['segmentation_immune_cell_intensity_filtered'].compute())
io.imsave(os.path.join(masks_path, 'segmentation_mask_RBC.tiff'), sdata.labels['segmentation_RBC_cell_intensity_filtered'].compute())

### sdata

In [ ]:
sdata